In [ ]:
# Importing dependecies
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

<h2>Reflecting Tables into SQLAlchemy ORM</h2>

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
 # creating engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()

In [ ]:
# reflecting an existing database into a new model
Base=automap_base()
# reflecting the tables
Base.prepare(engine, reflect=True)

In [ ]:
# Viewing all of the classes that automap found
Base.classes.keys()

In [ ]:
# Saving references to each table
Measurement=Base.classes.measurement
Station=Base.classes.station

In [ ]:
# Creating our session (link) from Python to the DB
session=Session(engine)

In [ ]:
#Inspecting columns in the Measurement table
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
#Inspecting columns in the Station table
inspector = inspect(engine)
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

<h2> Exploratory Precipitation Analysis</h2>

In [ ]:
# Find the most recent date in the data set.
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

<h3>Designing a query to retrieve the last 12 months of precipitation data and plotting the results.</h3> 

In [ ]:
# Starting from the most recent data point - 2017-08-23 - in the database. 
#Using the most recent date to retrieve the last 12 months of precipitation data by querying the 12 preceding months of data
date = dt.datetime(2016,8,23)
annual_data = session.query(Measurement.date).\
    filter(Measurement.date > date).\
    order_by(Measurement.date).all()
annual_data

In [ ]:
# Calculating the date one year from 2017-08-23
year_ago = dt.date(2017,8,23) - dt.timedelta(days=365)
year_ago

In [ ]:
# Performing a query to retrieve the data and precipitation scores
#Selecting only the date and prcp values.
results = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date <= '2017-08-23').\
    filter(Measurement.date >= '2016-08-24').\
    order_by(Measurement.date).all()
results

In [ ]:
##Saving the query results as a Pandas DataFrame
# Using Pandas `read_sql_query` to load a query statement directly into the DataFramestmt 
stmt = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date > '2016-08-23').statement
df = pd.read_sql_query(stmt, session.bind)
df.head(10)

In [ ]:
#setting the index to the date column
df2 = df.set_index(keys='date')
df2.head()

In [ ]:
# Sortting the dataframe by date
df2=df2.sort_values(by=['date'])
df2.head()

In [ ]:
# Using Pandas Plotting with Matplotlib to plot the data
df2.plot(title="Percipitation over 12 months", alpha=0.75, rot=90).set_ylabel('Inches')
plt.show()

In [ ]:
# Using Pandas to calcualte the summary statistics for the precipitation data
df2.describe()

<h2> Exploratory Station Analysis</h2>

In [ ]:
# Designing a query to calculate the total number stations in the Station dataset
stations_count = session.query(Station).group_by(Station.station).count()
print(stations_count)

In [ ]:
# Designing a query to calculate the total number stations in the Measurement dataset
stations_count2 = session.query(Measurement).group_by(Measurement.station).count()
print(stations_count)

In [ ]:
# Designing a query to find the most active stations (i.e. what stations have the most rows?)
# Listing the stations and the counts in descending order.
station_activity = engine.execute("SELECT DISTINCT station, COUNT(prcp) AS prcp_count FROM measurement GROUP BY station ORDER BY prcp_count DESC").all()
for record in station_activity:
    print(record)

In [ ]:
most_active =  engine.execute("SELECT DISTINCT station, COUNT(prcp) AS prcp_count FROM measurement GROUP BY station ORDER BY prcp_count DESC").first()
most_active[0]

In [ ]:
# Usinging the most active station id from the previous query, calculate the lowest, highest, and average temperature.
#Finding the lowest temperature
min_temp = session.query(func.min(Measurement.tobs).label('Min_temp')).filter(Measurement.station==most_active[0])
for temp in min_temp:
    print(temp)

In [ ]:
#Finding highest temeparture
max_temp = session.query(func.max(Measurement.tobs).label('Max_temp')).filter(Measurement.station== most_active[0])
for temp in max_temp:
    print(temp)

In [ ]:
#Finding average temeparture
avg_temp = session.query(func.avg(Measurement.tobs).label('Avg_temp')).filter(Measurement.station== most_active[0])
for temp in avg_temp:
    print(temp)

In [ ]:
# Using the most active station id
# Querying the last 12 months of temperature observation data for this station
annual_temp = session.query(Measurement.date, Measurement.tobs).\
    filter(Measurement.date > '2016-08-23').\
    filter(Measurement.station==most_active[0]).\
    order_by(Measurement.tobs).all()
annual_temp

In [ ]:
#plotting the results as a histogram
#Unpacking tuples using list comprehensions
dates = [result[0] for result in annual_temp]
tobs = [int(result[1]) for result in annual_temp]
#plotting
plt.hist(tobs, density=True, bins=12)
plt.ylabel('Frequency')
plt.xlabel('Temperature');

<h2> Close Session </h2>

In [ ]:
 # Close Session
session.close()